# FACIAL RECOGNITION
## Digital Signal and Image Management Project 2021-2022<br/>

Matteo Cesaro - 867350 - m.cesaro1@campus.unimib.it<br/>
Francesco Martinelli - 873685 - f.martinelli21@campus.unimib.it<br/>
Cristiano Ruttico - 809360 - c.ruttico@campus.unimib.it<br/>

### Importazione Librerie
Si installano ed importano le librerie necessarie

In [ ]:
#!pip install scikit-learn

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.models import model_from_json
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [14]:
import numpy as np
import os
import cv2 as cv
import matplotlib.pyplot as plt
from skimage import data, color
from skimage.transform import resize

face_detector = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
ImageDataGenerator_obj = ImageDataGenerator(brightness_range=[0.5,0.5])

### Caricamento del Modello

Il modello viene caricato con i propri pesi addestrati sul dataset di immagini degli autori.

In [15]:
#creazione cartella
path = os.getcwd()
print ("The current working directory is %s" % path)

The current working directory is /Users/francescomartinelli


In [16]:
json_file = open('/Users/francescomartinelli/dsim/model_finale-2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/Users/francescomartinelli/dsim/model_finale-2.h5")
print("Loaded model from disk")

Loaded model from disk


In [17]:
identifier = loaded_model

Si controlla che il modello sia stato caricato correttamente. Come si vede non è presente il layer di input.

In [18]:
identifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 293, 293, 64)      9472      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 146, 146, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 144, 144, 32)      18464     
                                                                 
 conv2d_2 (Conv2D)           (None, 142, 142, 16)      4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 71, 71, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 80656)             0

In [19]:
model = Sequential()
model.add(Input(shape=(299,299,3)))
model.add(identifier)

Il layer di input viene aggiunto con shape identica a quella su cui è stato precedentemente addestrato il modello.

### Demo

Viene construita una funzione che riassume il preprocessing eseguito sulle immagini del dataset prima del training del modello e la predizione adoperata dal modello. La funzione chiamata "process_frame" prende in input un immagine e restituisce un colore. Ad ogni colore corrisponde una classe, come indicato nello script seguente. Se nessun volto è identificato il colore in output è il rosso (siamo in spazio BGR).

In [20]:
def process_frame(img):
    if who is not None:
        xyzh = face_detector.detectMultiScale(who[:,:,1])
        if len(xyzh) >= 1:
            xyzh = xyzh[0]
            face = who[xyzh[1]:(xyzh[1]+xyzh[0]), xyzh[0]+15:xyzh[0] + xyzh[2]-15]
            face = resize(face, (299,299,3), mode="symmetric", preserve_range=True)
            face = image.img_to_array(face)  
            face = np.expand_dims(face, 0)
            iterator = ImageDataGenerator_obj.flow(face, batch_size=1)
            chunk = iterator.next()
            face = chunk[0].astype('float32')/255
            face = np.expand_dims(face,0)
            pred = model.predict(face)
            pred = np.argmax(pred)
            if pred == 0:
                col = (0,255,255) # Yellow - 0 - Cristiano
            elif pred == 1:
                col = (0,255,0) # Green - 1 - Francesco
            elif pred == 2:
                col = (255,0,0) # Blue - 2 - Matteo
        else: 
            col = (0,0,255)
    
        return col

Si imposta lo streaming di frame. È stato aggiunto al codice utilizzato per la raccolta dati, la funzione " cv.circle" che traccia sull'immagine mostrata a schermo un cerchio nell'angolo in alto a sinistra. Il colore del cerchio di default è rosso ma premendo "p" il frame viene processato dalla funzione definita in precedenza e il colore del cerchio viene riassegnato in base all'output del modello. Premendo "q" come in precedenza si interrompe lo streaming.

In [36]:
# Streaming
col = (0,0,255)
cap = cv.VideoCapture(0)
while(True):
    # Capture new frame
    r, frame = cap.read()
    
    # Create figure
    cv.circle(frame, (100, 50), 20, col, 3)

    # Visualize (external window)
    cv.imshow('Video', frame)
    if cv.waitKey(20) & 0xFF == ord('q'): # Interrupt when Q key is pressed   
        break
    elif cv.waitKey(20) & 0xFF == ord('p'):
        who = frame
        col = process_frame(who)
        
 
cv.destroyAllWindows()
cap.release()


In [23]:
cap.release()
cv.destroyAllWindows()